In [308]:

using LinearAlgebra
using GenericLinearAlgebra


In [309]:
function complete_orthonormal(U::Matrix{Float64})
    d, r = size(U)
    r ≤ d || error("U deve ter no máximo d colunas")

    r == d && return zeros(Float64, d, 0)   # já completa

    # Projeta vetores aleatórios no complemento de span(U)
    Z = randn(Float64, d, d - r)
    Z .-= U * (U' * Z)                      # remove componente em U

    # QR fino garante ortonormalidade
    C = Matrix(qr(Z).Q)                  # d × (d−r)
    return C
end

complete_orthonormal (generic function with 2 methods)

In [310]:
function build_csd_input_matrix(D1::Matrix{Float64}, D2::Matrix{Float64})
    # Dimensões
    m, d = size(D1)  # D1: m x d
    n, _ = size(D2)  # D2: n x d

    # Passo 1: QR de D1^T para obter base de R(D1) em R^d
    Q1 = Matrix(qr(D1').Q)  # Q1 é d x m
    R1 = Matrix(qr(D1').R)  # R1 é m x m

    println("Q1: ", size(Q1))  # Debugging: verificar tamanho de Q1
    println("R1: ", size(R1))  # Debugging: verificar tamanho de R1

    r1 = sum(abs.(diag(R1)) .> eps() * maximum(abs.(diag(R1))))  # Estimar posto
    E1 = Q1[:, 1:r1]  
    E2 = complete_orthonormal(E1)  # Base ortonormal completa para R^d, d x (d - r1)

    # Passo 2: QR de D2^T para obter base de R(D2) em R^d
    Q2 = Matrix(qr(D2').Q)  # Q2 é d x n
    R2 = Matrix(qr(D2').R)  # R2 é n x n

    println("Q2: ", size(Q2))  # Debugging: verificar tamanho de Q2
    println("R2: ", size(R2))  # Debugging: verificar tamanho de R2

    c1 = sum(abs.(diag(R2)) .> eps() * maximum(abs.(diag(R2))))  # Estimar posto
    F1 = Q2[:, 1:c1]  
    F2 = complete_orthonormal(F1)  # Base ortonormal completa para R^d, d x (d - c1)

    # Passo 3: Calcular Q11 = E1^T F1
    Q11 = E1' * F1  # r1 x c1
    Q12 = E1' * F2  # r1 x (d - c1)
    Q21 = E2' * F1  # (d - r1) x c1
    Q22 = E2' * F2  # (d - r1) x (d - c1)

    # Passo 4: Concatenar Q11, Q12, Q21, Q22
    Qtop = hcat(Q11, Q12)
    Qbot = hcat(Q21, Q22)
    Q = vcat(Qtop, Qbot)
    
    return Q, r1, c1, E1, F1, R1, R2
end

build_csd_input_matrix (generic function with 1 method)

In [311]:
# Exemplo com dados aleatórios
m = 100  # amostras de D1
n = 100  # amostras de D2
d = 784  # número de features

D1 = randn(m, d)  # dataset 1
D2 = randn(n, d)  # dataset 2

Q, r1, c1, E1, F1, R1, R2 = build_csd_input_matrix(D1, D2)

Q1: (784, 100)
R1: (100, 100)
Q2: (784, 100)
R2: (100, 100)


([-0.004395828887589161 0.026246049017214022 … -0.022861230308363505 0.0022516150596950087; 0.007241232556139617 0.026672116601587054 … 0.032731656253628144 -0.0023310756726590536; … ; -0.0346925316881511 -0.03897390650149207 … 0.008391624359081017 -0.007143344560568445; -0.03086956028926277 0.014543246921978392 … 0.010568326539043243 -0.0284460691613042], 100, 100, [-0.04675970274134977 -0.008620465858658312 … -0.037644401855438984 -0.03036258783237923; -0.0026580440307741662 -0.0404767927993297 … -0.006723231031346106 -0.0066344801386137; … ; -0.016553998527005256 0.01568459951932759 … 0.09841387220110148 -0.0067032370044251945; 0.012891028506355439 0.011331476964166238 … -0.008709837820757751 0.03874725089808202], [-0.06007508971969666 0.0029311341936095053 … -0.056590304714505456 0.00660742552488137; 0.041295183275355994 -0.01060620029208903 … -0.07329281108627204 0.03281478259602441; … ; 0.025588892027770607 -0.05436209972995566 … 0.05808721646777409 -0.07892818651419094; -0.04128

In [312]:
println(size(Q))  # Verifica o tamanho da matriz Q
println("Posto de D1: ", r1)
println("Posto de D2: ", c1)
println("Tamanho de E1: ", size(E1))  # Verifica o tamanho de E1
println("Tamanho de F1: ", size(F1))  # Verifica o tamanho de E2
println("Tamanho de R1: ", size(R1))  # Verifica o tamanho de R1
println("Tamanho de R2: ", size(R2))  # Verifica o tamanho de R2

(784, 784)
Posto de D1: 100
Posto de D2: 100
Tamanho de E1: (784, 100)
Tamanho de F1: (784, 100)
Tamanho de R1: (100, 100)
Tamanho de R2: (100, 100)


In [313]:
# Verificar se R1' * E1' é aproximadamente igual a D1
if R1'*E1' ≈ D1
    println("R1' * E1' é aproximadamente igual a D1")
else
    println("R1' * E1' é diferente de D1")
end

R1' * E1' é aproximadamente igual a D1


In [314]:
function rebuild_datasets_matrices(E, R)
    D_reconstructed = R' * E'  
    # Retornar a matriz reconstruída
    return D_reconstructed
end

rebuild_datasets_matrices (generic function with 4 methods)

In [315]:
function verify_Q(Q::Matrix{Float64})
    if Q'*Q ≈ I
        println("A matriz Q é ortonormal")
    else
        println("A matriz Q não é ortonormal")
        
    end
end

verify_Q (generic function with 1 method)

In [316]:
# Exibir e verificar a corretude da matriz A
verify_Q(Q)
D1_reconstructed = rebuild_datasets_matrices(E1, R1)
D2_reconstructed = rebuild_datasets_matrices(F1, R2)

if D1_reconstructed ≈ D1 && D2_reconstructed ≈ D2
    println("As matrizes reconstruídas são equivalentes às originais!")
else
    println("As matrizes reconstruídas NÃO são equivalentes às originais.")
        
end

A matriz Q é ortonormal
As matrizes reconstruídas são equivalentes às originais!
